# Class 4 - Tree-based models

In [ ]:
#!pip install pandas_profiling

In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None 

**Downloading and pre-processing dataset**

We'll use IMDB 5000 Movies dataset in the analysis

In [ ]:
dataset = pd.read_csv("IMDB.csv")
dataset.head()

Comprehensive data report may be generated using `pandas_profiling` package - **IMDB_Report.zip** contains profiling report for IMDB dataset. More information about the package can be found on [Pandas-profiling official docs](https://pandas-profiling.ydata.ai/docs/master/)

In [ ]:
profile = ProfileReport(dataset, title = "IMDB Movies Profiling Report")

In [ ]:
profile.to_file(output_file = "IMDB_Report.html")

In [ ]:
#Inspecting columns
dataset.columns

In [ ]:
#Checking numeric columns
dataset.describe(include = [np.number])

In [ ]:
#Checking nominal columns
dataset.describe(include = ['O']) 

In [ ]:
#Dropping columns with many unique values and imbalanced classes
dataset.drop(['color', 'director_name', 'actor_2_name', 'actor_1_name',
             'movie_title', 'actor_3_name', 'plot_keywords',
             'movie_imdb_link', 'language', 'country', 'content_rating'],
             axis = 1, inplace = True)

In [ ]:
#Drop duplicates
print(dataset.shape)
dataset.drop_duplicates(inplace = True)
print(dataset.shape)

In [ ]:
#Check null values
dataset.isnull().sum()

In [ ]:
#Dropping missing values
dataset.dropna(inplace = True)
dataset.shape

**Initial EDA (Exploratory Data Analysis)**

In [ ]:
numeric_dataset = dataset.drop("genres", axis = 1)

In [ ]:
f, axes = plt.subplots(4, 4, figsize = [15, 15])
plt.tight_layout(pad = 0.4, w_pad = 1.0, h_pad = 1.0)
for n,col in enumerate(numeric_dataset):
    sns.regplot(x = col, y = "imdb_score", data = dataset, ax = axes[n//4, n%4])

In [ ]:
f, axes = plt.subplots(4, 4, figsize = [15,15])
plt.tight_layout(pad = 0.4, w_pad = 1.0, h_pad = 1.0)
for n,col in enumerate(numeric_dataset):
    sns.boxplot(x = col, data = dataset, ax = axes[n//4, n%4])

**Detecting outliers**

A raw score x is converted into a standard score by

$$ z= \frac{x-\mu}{\sigma}  $$

where:

* μ is the mean of the population,
* σ is the standard deviation of the population.

In [ ]:
stats.zscore(numeric_dataset)

In [ ]:
#Removing outliers
dataset = dataset[(np.abs(stats.zscore(numeric_dataset)) < 9).all(axis = 1)]
dataset.shape

**Feature engineering**

In [ ]:
# Splitting genres column values
dataset['genres'] = dataset.genres.str.split("|")
dataset['genres']

In [ ]:
# Getting distinct categories
categories = set(dataset.genres.explode())
categories

In [ ]:
# One-hot encode each movie's classification
for cat in categories:
    dataset[cat] = dataset.genres.apply(lambda s: int(cat in s))
dataset.head()

In [ ]:
#Drop genres column
dataset.drop('genres', axis = 1, inplace = True)

**EDA on cleaned data**

In [ ]:
f, axes = plt.subplots(4, 4, figsize = [15, 15])
plt.tight_layout(pad = 0.4, w_pad = 1.0, h_pad = 1.0)
for n,col in enumerate(dataset.columns[0:16]):
    sns.regplot(x = col, y = "imdb_score", data = dataset, ax = axes[n//4, n%4])

In [ ]:
f, axes = plt.subplots(6, 4, figsize = [15, 15])
plt.tight_layout(pad = 0.4, w_pad=1.0, h_pad = 1.0)
for n, col in enumerate(dataset.columns[16:]):
    sns.barplot(x = col, y = "imdb_score", data = dataset, ax = axes[n//4, n%4]).set_ylim(5,)
f.delaxes(axes[5, 2]); f.delaxes(axes[5, 3]);

**Splitting data into train and test subsets**

In [ ]:
X = dataset.drop('imdb_score', axis = 1)
y = dataset['imdb_score']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.15, random_state = 42)

**Decision Trees**

From: [sklearn docs](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart)

_What are all the various decision tree algorithms and how do they differ from each other? Which one is implemented in scikit-learn?_

**ID3** (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

**C4.5** is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

**C5.0** is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

**CART** (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.

**scikit-learn uses an optimised version of the CART algorithm**

CART algorithm pick variables and cutoff threshold using:
 1. __for classification__: minimization of node's heterogeneity (Gini index or entropy) 
 2. __for regression__: minimizing error of prediction (e.g. sum of squares of residuals)

In [ ]:
from sklearn import tree
from matplotlib.pyplot import figure

In [ ]:
CART = tree.DecisionTreeRegressor(random_state = 42, ccp_alpha = 0.0)
CART_model = CART.fit(X_train, y_train)

In [ ]:
CART_model.get_depth()

In [ ]:
CART_model.get_n_leaves()

**Pruning CART tree (cost based)**

[Minimal Cost-Complexity Pruning](https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning)

In [ ]:
path = CART.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas[::5], path.impurities[::5]
figure(figsize = (12, 8))
plt.plot(ccp_alphas[:-1], impurities[:-1], marker = 'o', drawstyle = "steps-post")
plt.xlabel("Cost parameter", fontsize = 15)
plt.ylabel("Total impurity of leaves", fontsize = 15)
plt.title("Total Impurity vs complexity hyperparameter for training set", fontsize = 20);

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = tree.DecisionTreeRegressor(random_state = 42, ccp_alpha = ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)

In [ ]:
for order, ind in [('First', 0), ('Last', -1)]:
    print(f"{order} tree with ccp_alpha: {ccp_alphas[ind]:.2f}, " +
          f"nodes: {clfs[ind].tree_.node_count}, leaves: {clfs[ind].get_n_leaves()}")

In [ ]:
def RMSE(model, X, y):
    return np.sqrt(((model.predict(X) - y)**2).mean())

In [ ]:
test_scores = [RMSE(clf, X_test, y_test) for clf in clfs]
train_scores = [RMSE(clf, X_train, y_train) for clf in clfs]

fig, ax = plt.subplots(figsize = [12, 8])
ax.set_xlabel("Alpha", fontsize = 15)
ax.set_ylabel("RMSE", fontsize = 15)
ax.set_title("RMSE vs alpha for training and test sets", fontsize = 20)
ax.plot(ccp_alphas, train_scores, marker = 'o', label = "train", drawstyle = "steps-post")
ax.plot(ccp_alphas, test_scores, marker = 'o', label = "test", drawstyle = "steps-post")
ax.legend()
plt.show()

In [ ]:
#Complexity (cost) that produce the best regression tree
Best_CART = clfs[np.argmin(test_scores)]
Best_CART.ccp_alpha

In [ ]:
#Visualize the tree
plt.figure(figsize = (15, 10))
_ = tree.plot_tree(Best_CART, #clfs[-1] 
                   feature_names = X_train.columns,  
                   filled = True)

In [ ]:
#RMSE of the best tree
min(test_scores)

In [ ]:
#But we can view this as multiclass classification
confmat = pd.crosstab(Best_CART.predict(X_test).round(), y_test.round())
confmat

In [ ]:
#Accuracy
np.array([confmat.loc[i,i] for i in confmat.index]).sum()/confmat.sum().sum()

### Ensemble tree-based methods

[Ensemble learning](https://scikit-learn.org/stable/modules/ensemble.html) helps improve final model performance by combining results of underlying models (e.g. random forest is combination of decision trees).

Two families of ensemble methods are usually distinguished:

- In **averaging methods**, the main principle is to build several estimators **independently** and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator.

> Example: Random forests

- By contrast, in **boosting methods**, base estimators are built **sequentially** and the following models tries to reduce the error of the combined estimator.

> Example: Boosted trees

<img src="https://hpccsystems.com/wp-content/uploads/2022/09/LearningTrees-1.png" width=500>

[Source](https://hpccsystems.com/resources/learning-trees-a-guide-to-decision-tree-based-machine-learning/)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

**Random forests**

https://scikit-learn.org/stable/modules/ensemble.html#random-forests

Random forest is a collection of 'weak' decision trees providing good performance together.

Trees are weakned using multiple techniques:
* bootstrap sample, potentially on subset of available data
* limiting number of features
* no pruning

We'll look on two parameters in `RandomForestRegressor` class:
- n_estimators - number of trees built in ensemble
- max_features - how many features will be included in each tree e.g.
    - 'auto' - all
    - 'sqrt' - random sample of sqrt(n_features)
    - n - random sample of 'n' features

In [ ]:
#Checking number of tress influence on RMSE
rfr = RandomForestRegressor
N = [10, 50, 100, 200, 300, 400, 500]
RMSE_RF= [RMSE(rfr(n, n_jobs = -1).fit(X_train, y_train), X_test, y_test) for n in N]

In [ ]:
figure(figsize = (12, 8))
plt.plot(N, RMSE_RF, '.-', color = 'g');
plt.xlabel("# trees", fontsize = 15)
plt.ylabel("RMSE", fontsize = 15)
print("Minimum for", N[np.argmin(RMSE_RF)], "trees")

In [ ]:
#Checking number of features influence on RMSE
features = np.linspace(1, X_train.shape[1], 10).astype(int)
RMSE_RF_features= [RMSE(rfr(400, max_features = n, n_jobs = -1).fit(X_train, y_train), X_test, y_test) for n in features]

In [ ]:
figure(figsize = (12, 8))
plt.plot(features, RMSE_RF_features, '.-', color = 'r');
features[np.argmin(RMSE_RF_features)]
plt.xlabel("# features", fontsize = 18)
plt.ylabel("RMSE", fontsize = 18)
print("Minimum for", features[np.argmin(RMSE_RF_features)], "features")

**How to handle the uncertainty in picking optimal values of hyperparameters?**

[Cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html)

![](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

In [ ]:
Best_RF = RandomForestRegressor(400, max_features = 25, n_jobs = -1).fit(X_train, y_train)

In [ ]:
# Plot the feature importances of the forest
importances = Best_RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in Best_RF.estimators_], axis = 0)
indices = np.argsort(importances)[::-1]

num_feat = 6
plt.figure(figsize = [15, 8])
plt.title("Feature importances", fontsize = 20)
plt.bar(range(num_feat)[:num_feat], importances[indices][:num_feat],
       color = "r", yerr = std[indices][:num_feat], align = "center")
plt.xticks(range(num_feat)[:num_feat], X_train.columns[indices[:num_feat]])
plt.xlim([-1, num_feat])
plt.ylabel("Impurity reduction", fontsize = 15)
plt.show()

[**Gradient Boosted Trees**](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting)

In [ ]:
#Checking number of tress influence on RMSE
gbr = GradientBoostingRegressor
N = [10, 50, 100, 200, 300, 400, 500, 600, 700]
RMSE_GBT = [RMSE(gbr(n_estimators = n).fit(X_train, y_train), X_test, y_test) for n in N]

figure(figsize = (12, 8))
plt.plot(N, RMSE_GBT, '.-', color = 'r');
plt.xlabel("# trees", fontsize = 18)
plt.ylabel("RMSE", fontsize = 18)
print("Minimum for", N[np.argmin(RMSE_GBT)], "trees")

In [ ]:
#From 500 trees RMSE reduction is insignificant
Best_GBT = GradientBoostingRegressor(n_estimators = 500).fit(X_train,y_train)

In [ ]:
# Plot feature importance
feature_importance = Best_GBT.feature_importances_
# Make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
num_feat = 6

plt.figure(figsize = [12, 8])
plt.barh(pos[-num_feat:], feature_importance[sorted_idx][-num_feat:], align='center', alpha = 0.75)
plt.yticks(pos[-num_feat:], X_train.columns[sorted_idx][-num_feat:])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

**Comparing results of Decision Tree, Random Forest and Gradient Boosted Trees**

In [ ]:
models = [Best_CART, Best_RF, Best_GBT]
errors = [RMSE(m, X_test, y_test) for m in models]

In [ ]:
plt.bar(['CART','Random Forest','Gradient Boosted Trees'], errors, color = ['red', 'green', 'blue'], alpha = 0.75)
plt.ylabel('RMSE');

In [ ]:
# from sklearn.metrics import mean_squared_error